In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

service = Service(ChromeDriverManager().install())

options = Options()
options.add_argument("--headless")
options.add_argument("--window-size=1920x1080")
options.add_argument("--lang=ko_KR")
options.add_argument("--start-maximized")
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36 Edg/138.0.0.0")

driver = webdriver.Chrome(service=service, options=options)

url = "https://tickets.interpark.com/contents/genre/concert"
driver.get(url)
time.sleep(2)

try : 
    hot_button = driver.find_element(By.XPATH, "//button[contains(text(), '요즘 HOT')]")
    print(hot_button)
    hot_button.click()
    time.sleep(2)
    
except Exception as e:
    print("요즘 HOT 버튼 클릭 실패 : ", e)

scroll_pause = 1
last_height = driver.execute_script("return document.body.scrollHeight")

for _ in range(5) :
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height :
        break
    last_height = new_height

titles = driver.find_elements(By.CSS_SELECTOR, ".TicketItem_goodsName__Ju76j")
dates = driver.find_elements(By.CSS_SELECTOR, ".TicketItem_playDate__5ePr2")
places = driver.find_elements(By.CSS_SELECTOR, ".TicketItem_placeName__ls_9C")

data = []

for t, d, p in zip(titles, dates, places) :
    data.append({
        "title" : t.text.strip(),
        "date" : d.text.strip(),
        "place" : p.text.strip(),
    })

# print(data)

driver.quit()

df = pd.DataFrame(data)
df.head()

<selenium.webdriver.remote.webelement.WebElement (session="973a8d5c6f9bf001d1521cb787cca3e1", element="f.96AEE59D8D66FEB60D280636AD1339BF.d.D868ED0D56385A85ADE31396161B7019.e.44")>


,title,date,place
0,변진섭 전국투어 콘서트 : 변천사 시즌2,2025.02.09 -,전국 각 지역
1,RETRO MUSIC CONCERT 〈최재훈×플라워〉,"2025.08.09, 2025.10.19","서울,인천"
2,옥탑재즈,2025.02.14 - 2025.10.24,아트센터인천 다목적홀
3,2025 심수봉 전국투어 콘서트：꽃길 - 안동,2025.09.13,안동문화예술의전당 웅부홀
4,홍경민 매달리스트 콘서트,2025.08.02,SA HALL


In [8]:
pip install pymysql

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [16]:
import pymysql

conn = pymysql.connect(
    host = 'localhost', 
    user = 'root',
    password = '753951DnDn!!',
    db = 'interparkmkt',
    charset = 'utf8mb4'
)

cursor = conn.cursor()

for i, row in df.iterrows():
    sql = """
    INSERT INTO performances (title, data_range, place)
    VALUES (%s, %s, %s)
    """
    cursor.execute(sql, (row["title"], row["date"], row["place"]))

conn.commit()
conn.close()